In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time

from symbol import Symbol
from helpers import rename_columns, SafeHDFStore

In [2]:
s = Symbol()
symbol_list = s.symbol_list
print(len(symbol_list))

1504


In [4]:
driver = webdriver.Firefox()
driver.get('https://www.edelweiss.in/quotes/equity/Infosys-Ltd-2806')

The slowest run took 4.62 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 4.44 s per loop


In [5]:
edelweiss_list =  {
    'federalbnk': 'https://www.edelweiss.in/quotes/equity/Federal-Bank-Ltd-3709',
    '20micons': 'https://www.edelweiss.in/quotes/equity/20-Microns-Ltd-23450',
    '8kmiles': 'https://www.edelweiss.in/quotes/equity/8K-Miles-Software-Services-Ltd-24603',
    'tatamotors': 'https://www.edelweiss.in/quotes/equity/Tata-Motors-Ltd-560'
    
}


In [97]:
ratios = {
    '0': 'margin_ratio_consolidated',
    '1': 'margin_ratio_standalone',
    '2': 'return_ratio_consolidated',
    '3': 'return_ratio_standalone',
    '6': 'leverage_ratio_consolidated',
    '7': 'leverage_ratio_standalone',
    '8': 'turnover_ratio_consolidated',
    '9': 'turnover_ratio_standalone',
    '10': 'per_share_consolidate',
    '11': 'per_share_standalone',
    '12': 'valuation_ratio_consolidated',
    '13': 'valuation_ratio_standalone'
}

In [98]:
delay = 60
edelweiss_list = pd.DataFrame(
    index=symbol_list.index, columns=[
        'edelweiss_url', 'company_name', 'sector', 'pe_ratio', 'div_yield',
        'mcap', 'beta', 'bullish',
    ]
)

ratios_df = pd.DataFrame(columns=[
    'symbol', 'ratio_category', 'ratio_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
])

accounts_df = pd.DataFrame(columns=[
    'symbol', 'accounts_category', 'accounting_item_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
])

peers = pd.DataFrame(columns = [
    'symbol', 'rival_company', 'mcap', 'pe_ratio', 'pb_ratio',
    'ebidta', 'roce_pc', 'div_yield', 'debt_to_equity'
])

In [101]:
for symbol in symbol_list.index[2: 4]:
    try:
        search_box = driver.find_element_by_id('searchAnything')
        search_box.clear()
        search_box.send_keys(symbol)
        search_box.send_keys(Keys.ENTER)
        WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CLASS_NAME, 'highcharts-container')))
        
    except TimeoutException:
        print("Loading took too much time for {0}".format(symbol))
        continue
    
    time.sleep(3)
    dataframe_list = pd.read_html(driver.page_source)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    edelweiss_list.ix[symbol, 'edelweiss_url'] = driver.current_url
    edelweiss_list.ix[symbol, 'company_name'] = soup.find('h2', {'class': 'mg-t40 text-center secondheading'}).text
    edelweiss_list.ix[symbol, 'sector'] = soup.find('ul', {'class': 'list-inline text-center top-link-menu qt-sect-cont'}).a.text
    pe_div_yield = dataframe_list[1].copy()
    edelweiss_list.loc[symbol, 'pe_ratio'] = pe_div_yield.ix[:, 1][0]
    edelweiss_list.loc[symbol, 'div_yield'] = pe_div_yield.ix[:, 1][1]
    mcap_df = dataframe_list[2].copy()
    edelweiss_list.loc[symbol, 'mcap'] = mcap_df.ix[:, 1][0]
    edelweiss_list.loc[symbol, 'beta'] = mcap_df.ix[:, 1][1]
    try:
        edelweiss_list.loc[symbol, 'bullish'] = soup.find_all('div', {'class': 'bullish'})[0].span.text
    except:
        pass
    ratio_df_list = pd.read_html(str(soup.find('div', {'class': 'key-ratio top-brdr-block mg-t60 mg-x-t40', 'id': 'key-ratios'})))
    for ratio_order in ratios.keys():
        print(len(ratio_df_list), ratio_order)
        ratio_df = ratios_df_list[int(ratio_order)]
        if ratio_df.columns.size <= 1:
            continue
        ratio_df.columns = ratio_df.iloc[0]
        ratio_df = ratio_df.set_index(np.nan).drop(np.nan)
        ratio_df.index.name = 'ratio_name'
        ratio_df = ratio_df.reset_index()
        ratio_df.insert(0, 'symbol', symbol)
        ratio_df.insert(1, 'ratio_category', ratios[ratio_order])
        ratio_df = ratio_df.replace('-', np.nan)
        ratios_df = ratios_df.append(ratio_df).reset_index(drop=True)
    ratios_df = ratios_df[[
        'symbol', 'ratio_category', 'ratio_name', 'FY16',
        'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
    ]]
    accounts_df_list = pd.read_html(str(soup.find('div', {'class': 'key-ratio top-brdr-block mg-t60 mg-x-t40', 'id': 'financial-health'})))
    for account_order in financials.keys():
        account_df = accounts_df_list[int(account_order)].copy()
        if account_df.columns.size <= 1:
            continue
        account_df.columns = account_df.iloc[0]
        try:
            account_df = account_df.set_index(np.nan).drop(np.nan)
        except:
            continue
        account_df.index.name = 'accounting_item_name'
        account_df = account_df.reset_index()
        account_df.insert(0, 'symbol', symbol)
        account_df.insert(1, 'accounts_category', financials[account_order])
        account_df = account_df.replace('-', np.nan)
        accounts_df = accounts_df.append(account_df).reset_index(drop=True)
    accounts_df = accounts_df[[
        'symbol', 'accounts_category', 'accounting_item_name', 'FY16',
        'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
    ]]
    
    peer_comparision = pd.read_html(str(soup.find('div', {'class': 'top-brdr-block performance mg-t60 mg-x-t40 ng-scope', 'id': 'peer-performance'})))
    peer_comparision = peer_comparision[0]
    peer_comparision = peer_comparision.drop(0)
    peer_comparision.insert(0, 'symbol', symbol)
    print(peer_comparision)
    peer_comparision.columns = [
        'symbol', 'rival_company', 'mcap', 'pe_ratio', 'pb_ratio',
        'ebidta', 'roce_pc', 'div_yield', 'debt_to_equity'
    ]
    peers = peers.append(peer_comparision).reset_index(drop=True)
    element = soup.find_all('ul', {'class': 'list-unstyled patterns list-inline'})[0].find_all('li')
    edelweiss_list.loc[symbol, 'bse_identifier'] = element[0].text[5:]
    edelweiss_list.loc[symbol, 'nse_identifier'] = element[1].text[5:]
    edelweiss_list.loc[symbol, 'isin_identifier'] = element[2].text[6:]
    with SafeHDFStore('edelweiss.h5') as store:
        store.put('list', value=edelweiss_list)
        store.put('ratios', value=ratios_df)
        store.put('accounts', value=accounts_df)
        store.put('peers', value=peers)
    print('Successful for {0}'.format(symbol))

13 13


IndexError: list index out of range

In [103]:
pd.read_html(str(soup.find('div', {'class': 'key-ratio top-brdr-block mg-t60 mg-x-t40', 'id': 'key-ratios'})))

[                             0
 0      Gross Profit Margin (%)
 1  Operating Profit Margin (%)
 2        Net Profit Margin (%),
                              0      1      2     3     4     5
 0                          NaN   FY16   FY15  FY14  FY13  FY12
 1      Gross Profit Margin (%)  16.32  11.11  6.42  6.75  8.37
 2  Operating Profit Margin (%)  16.44  11.32  7.23  7.37  8.64
 3        Net Profit Margin (%)   9.19   5.62  2.36  3.16  4.40,
                                 0
 0            Return on Equity (%)
 1  Return on Capital Employed (%),
                                 0      1      2      3      4      5
 0                             NaN   FY16   FY15   FY14   FY13   FY12
 1            Return on Equity (%)  22.12  14.38   6.34   8.29  11.33
 2  Return on Capital Employed (%)  32.98  20.17  10.01  11.14  15.98,
     0     1     2     3     4     5
 0 NaN  FY16  FY15  FY14  FY13  FY12,
                           0
 0  Debt to Equity Ratio (x)
 1     Interest Coverage (x),


In [96]:


financials = {
    '0': 'annual_pl_standalone',
    '1': 'annual_pl_consolidated',
    '2': 'annual_bs_standalone',
    '3': 'annual_bs_consolidated',
    '4': 'annual_cf_standalone',
    '5': 'annual_cf_consolidated',
}
pd.read_hdf('edelweiss.h5', 'accounts').query('symbol == "8kmiles"')

,symbol,accounts_category,accounting_item_name,FY16,FY15,FY14,FY13,FY12,FY11
25,8kmiles,margin_ratio_consolidated,Cash flow from Operating Activities,10.54,1.37,1.30,-4.80,4.88,NaN
26,8kmiles,margin_ratio_consolidated,Cash flow from Investing Activities,-28.62,-11.27,-1.27,-1.65,-1.03,NaN
27,8kmiles,margin_ratio_consolidated,Cash flow from Financing Activities,16.01,14.99,-0.03,4.15,-1.55,NaN
28,8kmiles,margin_ratio_consolidated,Share Capital,10.89,10.34,10.04,10.04,5.55,NaN
29,8kmiles,margin_ratio_consolidated,Reserves,52.74,35.90,12.03,11.94,13.69,NaN
30,8kmiles,margin_ratio_consolidated,Borrowings,2.72,0.07,2.62,1.03,1.66,NaN
31,8kmiles,margin_ratio_consolidated,Other Liabilities,7.72,1.22,1.96,0.64,4.79,NaN
32,8kmiles,margin_ratio_consolidated,Total Liabilities,74.07,47.53,26.65,23.65,25.69,NaN
33,8kmiles,margin_ratio_consolidated,Fixed Assets,7.47,11.75,10.96,10.77,9.80,NaN
34,8kmiles,margin_ratio_consolidated,Other Assets,7.94,4.53,4.29,1.26,3.91,NaN


In [ ]:
lists_i = pd.read_html(driver.page_source)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
peer_comparision = lists_i[26].copy()
peer_comparision = peer_comparision.drop(0)
peer_comparision.insert(0, 'symbol', symbol)
peer_comparision.columns = ['symbol', 'rival_company', 'mcap', 'pe_ratio', 'pb_ratio', 'ebidta', 'roce_pc', 'div_yield', 'debt_to_equity']
peer_comparision

In [ ]:
lists_i = pd.read_html(driver.page_source)
soup = BeautifulSoup(driver.page_source, 'html.parser')
sector = soup.find_all('ul', {'class': 'list-inline text-center top-link-menu qt-sect-cont'})[0].a.text
pe_div_yield = lists_i[1]
pe_ratio = pe_div_yield.ix[:, 1][0]
div_yield = pe_div_yield.ix[:, 1][1]
mcap_df = lists_i[2]
mcap = mcap_df.ix[:, 1][0]
beta = mcap_df.ix[:, 1][1]
edel_bull = soup.find_all('div', {'class': 'bullish'})[0].span.text

ratios = {
    '6': 'margin_ratio_standalone',
    '7': 'margin_ratio_consolidated',
    '8': 'return_ratio_standalone',
    '9': 'return_ratio_consolidated',
    '12': 'leverage_ratio_standalone',
    '13': 'leverage_ratio_consolidated',
    '14': 'inventory_ratio_standalone',
    '15': 'inventory_ratio_consolidated',
    '16': 'per_share_standalone',
    '17': 'per_share_consolidated',
    '18': 'valuation_ratio_standalone',
    '19': 'valuation_ratio_consolidated',
}

data_df = pd.DataFrame(columns=[
    'symbol', 'ratio_category', 'ratio_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
])

for ratio_order in ratios.keys():
    ratio_df = lists_i[int(ratio_order)].copy()
    ratio_df.columns = ratio_df.iloc[0]
    ratio_df = ratio_df.set_index(np.nan).drop(np.nan)
    # TODO rename columns
    ratio_df.index.name = 'ratio_name'
    ratio_df = ratio_df.reset_index()
    ratio_df.insert(0, 'symbol', 'infy')
    ratio_df.insert(1, 'ratio_category', ratios[ratio_order])
    ratio_df = ratio_df.replace('-', np.nan)
    data_df = data_df.append(ratio_df).reset_index(drop=True)
data_df = data_df[[
    'symbol', 'ratio_category', 'ratio_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
]]
financials = {
    '20': 'annual_pl_standalone',
    '21': 'annual_pl_consolidated',
    '22': 'annual_bs_standalone',
    '23': 'annual_bs_consolidated',
    '24': 'annual_cf_standalone',
    '25': 'annual_cf_consolidated',
}

accounts_df = pd.DataFrame(columns=[
    'symbol', 'accounts_category', 'accounting_item_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
])

for account_order in financials.keys():
    account_df = lists_i[int(account_order)].copy()
    account_df.columns = account_df.iloc[0]
    account_df = account_df.set_index(np.nan).drop(np.nan)
    # TODO rename columns
    account_df.index.name = 'accounting_item_name'
    account_df = account_df.reset_index()
    account_df.insert(0, 'symbol', 'infy')
    account_df.insert(1, 'accounts_category', ratios[ratio_order])
    account_df = account_df.replace('-', np.nan)
    accounts_df = accounts_df.append(account_df).reset_index(drop=True)
accounts_df = accounts_df[[
    'symbol', 'accounts_category', 'accounting_item_name', 'FY16',
    'FY15', 'FY14', 'FY13', 'FY12', 'FY11'
]]
peer_comparision = lists_i[26]
peer_comparision = peer_comparision.drop(0)
peer_comparision.insert(0, 'symbol', 'infy')
peer_comparision = peer_comparision.rename(columns={'Company':'rival_company'})
element = soup.find_all('ul', {'class': 'list-unstyled patterns list-inline'})[0].find_all('li')
bse_identifier = element[0].text
nse_identifier = element[1].text
isin_identifier = element[2].text

In [ ]:

financials = {
    '20': 'annual_pl_standalone',
    '21': 'annual_pl_consolidated',
    '22': 'annual_bs_standalone',
    '23': 'annual_bs_consolidated',
    '24': 'annual_cf_standalone',
    '25': 'annual_cf_consolidated',
}
